In [155]:
import nltk
import random
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
import json
import pickle
from sklearn.metrics import accuracy_score, f1_score

In [141]:
label_map={'neg':0 ,'sli_neg':1 ,'neutral':2 ,'sli_pos':3 ,'pos':4}

In [142]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,label_map[each_file]) for sentence in data_temp]
    return dataset

In [143]:
def get_words(x):    #returns all the words in the  data set
    vocab = []
    for sentence,polarity in x:  
        vocab.extend(sentence.split())
    return vocab

In [144]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
vocab=set(get_words(labeled_data)) #getting words 

In [145]:
#bigrams
print labeled_data[:5]

[(u'learn way work corpor environ', 2), (u'job work life balanc salari benefit job secur advanc manag job cultur', 2), (u'perfect compani work life balanc allow work home', 4), (u'learn manag time', 2), (u'great work experi work ti', 4)]


In [146]:
print len(labeled_data)

201378


In [147]:
vectorizer = CountVectorizer(vocabulary=vocab,min_df=1)
corpus=[sentence for sentence,pol in labeled_data]
features=vectorizer.fit_transform(corpus[:60000])
labels=[label for sentence,label in labeled_data[:60000]]

In [148]:
classifier=svm.SVC()
classifier.kernel='linear'

In [149]:
classifier.fit(features[:25000],labels[:25000])
with open('svc_unigram_model','w') as f:
    pickle.dump(classifier,f) 

In [150]:
len(vocab)

6544

In [151]:
#accuracy
#with open('svc_unigram_model') as f:
#    classifier=pickle.load(f)
features_test=vectorizer.fit_transform(corpus[50000:55000])
predicted = classifier.predict(features_test)
accuracy=accuracy_score(labels[50000:55000],predicted)
#with open('accuracy_svc_unigram','w') as f:
#    json.dump(accuracy,f) 

In [166]:
accuracy

0.85199999999999998

In [164]:
accuracy_f1=f1_score(labels[50000:55000],predicted, average='weighted')

In [165]:
accuracy_f1

0.85138476388811901

In [ ]:
#testing
feature=[]
with open('svc_unigram_model') as f:
    classifier=pickle.load(f)
test=vectorizer.fit_transform(corpus[50001])
print test
#for word in vocab:
#    if word in corpus[60000].split():
#        feature.append(1)
#    else:
#        feature.append(0)
classifier.predict(test)

In [22]:
polarity=0
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))
    polarity=max(polarity,dist.prob(label))
print polarity

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931
0.585660041903
